## Without threads

In [1]:
import time 
start = time.perf_counter()
def do_something(time_to_run = 1):
    print(f'Sleeping {time_to_run} sec...')
    time.sleep(time_to_run)
    print('Done sleeping')

do_something(1.5)
do_something(1.5)
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1.5 sec...
Done sleeping
Sleeping 1.5 sec...
Done sleeping
Finished in 3.0 second(s)


## With threads

In [2]:
import threading 

start = time.perf_counter()
t1 = threading.Thread(target = do_something, args = [1.5])
t2 = threading.Thread(target = do_something, args = [1.5])
t1.start()
t2.start()

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1.5 sec...
Sleeping 1.5 sec...
Finished in 0.01 second(s)


Done sleeping
Done sleeping


## Waiting for threads to join

In [4]:
start = time.perf_counter()
t1 = threading.Thread(target = do_something ,args = [1.5] )
t2 = threading.Thread(target = do_something ,args = [1.5] )
t1.start()
t2.start()
t1.join()
t2.join()
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1.5 sec...
Sleeping 1.5 sec...
Done sleeping
Done sleeping
Finished in 1.52 second(s)


In [6]:
start = time.perf_counter()
threads = []
for _ in range(100):
    t = threading.Thread(target = do_something ,args = [1])
    t.start()
    threads.append(t)
for thread in threads:
    thread.join()
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...Sleeping 1 sec...

Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...Sleeping 1 sec...

Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1

## Concurrent futures

In [7]:
import concurrent.futures
start = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    for _ in range(10):
        executor.submit(do_something, 1)
# At the end of the 'with', we wait until all submitted processes complete.

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Sleeping 1 sec...
Done sleeping
Sleeping 1 sec...
Done sleeping
Sleeping 1 sec...
Done sleeping
Sleeping 1 sec...
Done sleepingDone sleeping

Sleeping 1 sec...
Sleeping 1 sec...
Done sleepingDone sleeping
Sleeping 1 sec...

Sleeping 1 sec...
Done sleeping
Sleeping 1 sec...
Done sleepingDone sleepingDone sleeping
Sleeping 1 sec...


Sleeping 1 sec...
Sleeping 1 sec...
Done sleeping
Sleeping 1 sec...
Done sleeping
Done sleepingDone sleeping

Done sleeping
Done sleepingDone sleeping

Done sleepingDone sleeping

Finished in 3.03 second(s)


In [41]:
def sum_list(thelist:list, start:int, end:int):
    s=0
    for i in range(start,end):
        s += thelist[i]**3//10
    return s


LIST_SIZE=3000000
big_list = list(range(LIST_SIZE))
start = time.perf_counter()
print("sum_list without threads: ", sum_list(big_list, 0, LIST_SIZE))
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} seconds')


sum_list without threads:  2024998650000224998650000
Finished in 2.04 seconds


In [43]:
start = time.perf_counter()

THREAD_COUNT=8
SUBLIST_SIZE = int(LIST_SIZE/THREAD_COUNT)
with concurrent.futures.ThreadPoolExecutor(THREAD_COUNT) as executor:
    futures = [executor.submit(sum_list, big_list, i*SUBLIST_SIZE, (i+1)*SUBLIST_SIZE) for i in range(THREAD_COUNT)]   # submit return a "future result".
    big_sum = 0
    for res in concurrent.futures.as_completed(futures):   # return each result as soon as it is completed:
        r = res.result()
        print("Partial sum: ", r)
        big_sum += r

print("Big sum: ", big_sum)
finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} seconds')

Big sum:  2024998650000224998650000
Finished in 2.49 seconds


## Race condition

In [44]:
import logging
import threading
import time
import concurrent.futures

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(1)   # simulates a delay during the computation
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)

start = time.perf_counter()

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.update, index)
logging.info("Testing update. Ending value is %d.", database.value)

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

19:29:31: Testing update. Starting value is 0.
19:29:31: Thread 0: starting update
19:29:31: Thread 1: starting update
19:29:32: Thread 0: finishing update
19:29:32: Thread 1: finishing update
19:29:32: Testing update. Ending value is 1.


Finished in 1.02 second(s)


## Locks

In [24]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(1)
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)

    def locked_update(self, name):
        logging.info("Thread %s: starting update", name)
        logging.debug("Thread %s about to lock", name)
        with self._lock:
            logging.debug("Thread %s has lock", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(1)
            self.value = local_copy
            logging.debug("Thread %s about to release lock", name)
        logging.debug("Thread %s after release", name)
        logging.info("Thread %s: finishing update", name)


logging.getLogger().setLevel(logging.DEBUG)
start = time.perf_counter()

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.locked_update, index)
logging.info("Testing update. Ending value is %d.", database.value)

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')


11:20:33: Testing update. Starting value is 0.
11:20:33: Thread 0: starting update
11:20:33: Thread 0 about to lock
11:20:33: Thread 0 has lock
11:20:33: Thread 1: starting update
11:20:33: Thread 1 about to lock
11:20:34: Thread 0 about to release lock
11:20:34: Thread 0 after release
11:20:34: Thread 1 has lock
11:20:34: Thread 0: finishing update
11:20:35: Thread 1 about to release lock
11:20:35: Thread 1 after release
11:20:35: Thread 1: finishing update
11:20:35: Testing update. Ending value is 2.


Finished in 2.04 second(s)
